In [2]:
import torch
import seaborn as sns
import sys
from pathlib import Path
sys.path.insert(0, "../..")
from src.data import make_dataset
from src.models import imagemodels
from src.models import train_model
from src.models import metrics
import gin

/home/azureuser/.cache/pypoetry/virtualenvs/poetry_add_pandas-Tn0MmM9i-py3.9/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-01-06 10:28:00.161684: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-06 10:28:00.309435: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-06 10:28:00.309459: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ign

In [3]:
gin.parse_config_file("model.gin")
train_dataloader, test_dataloader = make_dataset.get_MNIST()
accuracy = metrics.Accuracy()
# print(gin.config_str())

In [4]:

def hand_hypertuning(
    learning_rates,
    kernel_sizes,
    filters1,
    filters2,
    units1,
    units2,
    model_dic,
    epoch = 3,
    num_classes = 10,
    log_dir = "../../models/test/"):
  
    # make list if not list:
    def make_ls(x):
        return x if type(x) == type([]) else [x]

    # maak van loopers list if not:
    learning_rates = make_ls(learning_rates) 
    kernel_sizes = make_ls(kernel_sizes) 
    filters1 = make_ls(filters1) 
    filters2 = make_ls(filters2) 
    units1 = make_ls(units1) 
    units2 = make_ls(units2)

    #model naam en model object vullen:
    model_name = model_dic[0]
    
    
    #gin_config resetten en vullen:
    gin.clear_config()
    gin.parse_config_file("model.gin")
    
    gin.bind_parameter("trainloop.log_dir",log_dir)
    gin.bind_parameter("trainloop.epochs", epoch)
    gin.bind_parameter(model_name + '.num_classes', num_classes)
    # gin.bind_parameter(model_name + '.kernel_size', kernel_size)

    for fl1 in filters1:
        for fl2 in filters2:
            if fl2 <= fl1:
                for ut1 in units1:
                    for ut2 in units2:
                        if ut2 <= ut1:
                            for lr in learning_rates:
                                for kerz in kernel_sizes:
                                    
                                    # pm_dic = {
                                    #     fl1 : model_name + '.filter1'
                                    #     ,fl2 : model_name + '.filter2'
                                    #     ,ut1 : model_name + '.unit1'
                                    #     ,ut2 : model_name + '.unit2'
                                    #     ,kerz : model_name + '.kernel_size'
                                    #     ,lr : 'trainloop.learning_rate'
                                    # }
                                    # print(pm_dic)
                                    # for pm in pm_dic:
                                    #     print(pm)
                                    #     print(pm_dic[pm])
                                    #     gin.bind_parameter(pm_dic[pm],pm)


                                
                                    gin.bind_parameter(model_name + '.filter1', fl1)
                                    gin.bind_parameter(model_name + '.filter2', fl2)
                                    gin.bind_parameter(model_name + '.unit1', ut1)
                                    gin.bind_parameter(model_name + '.unit2', ut2)
                                    gin.bind_parameter(model_name + '.kernel_size', kerz)
                                    gin.bind_parameter("trainloop.learning_rate", lr)
                                    # print(gin.config_str())

                                    model = model_dic[1]

                                    model =  train_model.trainloop(
                                        model=model(),
                                        metrics=[accuracy],
                                        train_dataloader=train_dataloader,
                                        test_dataloader=test_dataloader,
                                        train_steps=len(train_dataloader),
                                        eval_steps=150,
                                    )
    
    return None

model_dic = {
    'CNN':{0:'CNN',1:imagemodels.CNN}
    ,'CNN_150': {0:'CNN_150',1:imagemodels.CNN_150}
    }
    


Als eerste wordt de learning rate getest:

In [6]:

hand_hypertuning(
    model_dic = model_dic['CNN']
    ,learning_rates = [0.01, 0.001, 0.0001]
    ,kernel_sizes = 2
    ,filters1 = 128
    ,filters2 = 128
    ,units1 = 128
    ,units2 = 64
    ,epoch = 10
    ,log_dir = "../../models/learning_rate/"
    )

2023-01-06 10:38:51.590 | INFO     | src.data.data_tools:dir_add_timestamp:114 - Logging to ../../models/learning_rate/20230106-1038
100%|██████████| 1875/1875 [01:17<00:00, 24.29it/s]
2023-01-06 10:40:11.806 | INFO     | src.models.train_model:trainloop:171 - Epoch 0 train 0.6659 test 0.5314 metric ['0.7990']
100%|██████████| 1875/1875 [01:17<00:00, 24.23it/s]
2023-01-06 10:41:31.977 | INFO     | src.models.train_model:trainloop:171 - Epoch 1 train 0.5046 test 0.5301 metric ['0.8113']
100%|██████████| 1875/1875 [01:17<00:00, 24.34it/s]
2023-01-06 10:42:51.782 | INFO     | src.models.train_model:trainloop:171 - Epoch 2 train 0.4659 test 0.5733 metric ['0.8042']
100%|██████████| 1875/1875 [01:18<00:00, 24.03it/s]
2023-01-06 10:44:12.581 | INFO     | src.models.train_model:trainloop:171 - Epoch 3 train 0.4629 test 0.4998 metric ['0.8260']
100%|██████████| 1875/1875 [01:16<00:00, 24.53it/s]
2023-01-06 10:45:31.941 | INFO     | src.models.train_model:trainloop:171 - Epoch 4 train 0.4569 te

KeyboardInterrupt: 

Nu we weten dat de optimale learning rate ligt op ongeveer 0.001 kan verder naar andere optimuns worden gezocht.
Als eerste wordt kernelsize getest:

Kernal; size is optimaal op 2 omdat filters en units de onformatie aan elkaar doorgeven, hebben filter zise en unitr een invloed op elkaar. De units nemen het over op een platten 128. dit geeft beperkingen deze zijn meegenomen:

In [7]:
fil_range_e3 = ([16,32,64,128],3)
fil_range_e10 = ([32,64],10)

for list in [fil_range_e3,fil_range_e10]:
    hand_hypertuning(
        model_dic = model_dic['CNN']
        ,learning_rates = 0.001
        ,kernel_sizes = 2
        ,filters1 = list[0]
        ,filters2 = list[0]
        ,units1 = 128
        ,units2 = 64
        ,log_dir = "../../models/filters/"
        ,epoch = list[1]
        )

2023-01-06 10:46:26.459 | INFO     | src.data.data_tools:dir_add_timestamp:114 - Logging to ../../models/filters/20230106-1046
100%|██████████| 1875/1875 [00:22<00:00, 82.06it/s]
2023-01-06 10:46:50.307 | INFO     | src.models.train_model:trainloop:171 - Epoch 0 train 0.7647 test 0.5769 metric ['0.7887']
100%|██████████| 1875/1875 [00:22<00:00, 83.24it/s]
2023-01-06 10:47:13.687 | INFO     | src.models.train_model:trainloop:171 - Epoch 1 train 0.5297 test 0.5126 metric ['0.8117']
100%|██████████| 1875/1875 [00:22<00:00, 83.03it/s]
2023-01-06 10:47:37.124 | INFO     | src.models.train_model:trainloop:171 - Epoch 2 train 0.4653 test 0.4550 metric ['0.8367']
100%|██████████| 3/3 [01:10<00:00, 23.51s/it]
2023-01-06 10:47:37.130 | INFO     | src.data.data_tools:dir_add_timestamp:114 - Logging to ../../models/filters/20230106-1047
100%|██████████| 1875/1875 [00:27<00:00, 68.41it/s]
2023-01-06 10:48:05.817 | INFO     | src.models.train_model:trainloop:171 - Epoch 0 train 0.7111 test 0.5777 me

KeyboardInterrupt: 

In [9]:
unit_range_e3 = ([16,32,64,128],3)
unit_range_e10 = ([32,64,128],10)

for list in [unit_range_e3,unit_range_e10]:
    hand_hypertuning(
        model_dic = model_dic['CNN']
        ,learning_rates = 0.001
        ,kernel_sizes = 2
        ,filters1 = 64
        ,filters2 = 64
        ,units1 = list[0]
        ,units2 = list[0]
        ,log_dir = "../../models/units/"
        ,epoch = list[1]
        )

2023-01-06 10:49:33.838 | INFO     | src.data.data_tools:dir_add_timestamp:114 - Logging to ../../models/units/20230106-1049
100%|██████████| 1875/1875 [00:42<00:00, 44.04it/s]
2023-01-06 10:50:18.313 | INFO     | src.models.train_model:trainloop:171 - Epoch 0 train 0.8018 test 0.6248 metric ['0.7708']
 33%|███▎      | 1/3 [00:59<01:59, 59.57s/it]


KeyboardInterrupt: 

In [10]:
fil_range_e3 = ([16,32,64,128],3)
fil_range_e10 = ([32,64],10)

for list in [fil_range_e3,fil_range_e10]:
    hand_hypertuning(
        model_dic = model_dic['CNN_150']
        ,learning_rates = 0.001
        ,kernel_sizes = 2
        ,filters1 = list[0]
        ,filters2 = list[0]
        ,units1 = 64
        ,units2 = 64
        ,log_dir = "../../models/filters_cnn150/"
        ,epoch = list[1]
        )

2023-01-06 10:51:04.307 | INFO     | src.data.data_tools:dir_add_timestamp:114 - Logging to ../../models/filters_cnn150/20230106-1051
100%|██████████| 1875/1875 [00:25<00:00, 72.47it/s]
2023-01-06 10:51:31.288 | INFO     | src.models.train_model:trainloop:171 - Epoch 0 train 0.8002 test 0.5733 metric ['0.7833']
100%|██████████| 1875/1875 [00:26<00:00, 72.10it/s]
2023-01-06 10:51:58.219 | INFO     | src.models.train_model:trainloop:171 - Epoch 1 train 0.4938 test 0.4948 metric ['0.8140']
100%|██████████| 1875/1875 [00:26<00:00, 71.08it/s]
2023-01-06 10:52:25.531 | INFO     | src.models.train_model:trainloop:171 - Epoch 2 train 0.4152 test 0.4027 metric ['0.8512']
100%|██████████| 3/3 [01:21<00:00, 27.02s/it]
2023-01-06 10:52:25.538 | INFO     | src.data.data_tools:dir_add_timestamp:114 - Logging to ../../models/filters_cnn150/20230106-1052
  0%|          | 0/3 [00:31<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
fil_unit_e3 = ([16,32,64,128,256],3)
fil_unit_e10 = ([32,64,128],10)

for list in [fil_unit_e3,fil_unit_e10]:
    hand_hypertuning(
        model_dic = model_dic['CNN_150']
        ,learning_rates = 0.001
        ,kernel_sizes = 2
        ,filters1 = 64
        ,filters2 = 32
        ,units1 = list[0]
        ,units2 = list[0]
        ,log_dir = "../../models/units_cnn150/"
        ,epoch = list[1]
        )

In [ ]:
%tensorboard